In [1]:
"""安装paddlex"""
! pip install paddlex==2.0.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 952kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 3.6MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 153kB 2.6MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 296kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 42.4MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 7.0MB/s eta 0:00:011
     |████████████████████████████████| 286kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
     |████████████████████████████████| 102kB 2.5MB/s ta 0:00:011
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.wh

In [1]:
"""解压B榜测试集"""
!unzip -oq data/data126175/dehw_testB_dataset.zip -d ./data

In [2]:
"""导入包"""
import paddlex as pdx
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
%matplotlib inline

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:136: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[01-21 01:16:38 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.1. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [3]:
'''
输入原始img_path。
第一步切分成n小块，将每个小块块喂给模型。
第二步，模型输出语义分割图，使用该语义分割图对结合原始图像进行后处理。
第三步，将模型所有的输出拼接在一起。
输出擦除手写文字的img。
'''
def erase_hand_write(img_path, model1):
    #读取图像，并切割
    img = Image.open(img_path)  #[H, W, C]
    img = np.array(img)
    img = img.transpose(2, 0, 1)    #[C, H, W]


    width, height = img.shape[2], img.shape[1]

    crop_img_list = img_crop(img)   #[C, H, W]
    process_img_list = []

    #使用模型输出每个切割图像的预测
    for i in range(len(crop_img_list)):
        process_img_list.append([])
        for j in range(len(crop_img_list[i])):
            img_path = Image.fromarray(crop_img_list[i][j].transpose(1, 2, 0).astype(np.uint8))
            img_path.save('tmp_img.png')
            predict_image = predict('tmp_img.png', model1)  #[H, W, C]
            process_img_list[i].append(predict_image)

    shape_y = len(process_img_list)
    shape_x = len(process_img_list[0])

    #拼接所有图像
    if shape_x == 1 and shape_y == 1:
        output_img = process_img_list[0][0]

    elif shape_x == 2 and shape_y == 1:
        output_img = np.zeros_like(img)     #[C, H, W]
        output_img = output_img.transpose([0, 2, 1])    #[C, W, H]
        
        output_img[:, 0:2048, :] = process_img_list[0][0].transpose(2, 1, 0)      #[C, W, H]
        output_img[:, width-2048:width, :] = process_img_list[0][1].transpose(2, 1, 0)      #[C, W, H]
        output_img = output_img.transpose(2, 1, 0).astype(np.uint8)

    elif shape_x == 1 and shape_y == 2:
        output_img = np.zeros_like(img)
        output_img = output_img.transpose([0, 2, 1])    #[C, W, H]
        output_img[:, :, :2048] = process_img_list[0][0].transpose(2, 1, 0)      #[C, W, H]
        output_img[:, :, height-2048:height] = process_img_list[1][0].transpose(2, 1, 0)      #[C, W, H]
        output_img = output_img.transpose(2, 1, 0).astype(np.uint8)

    else:
        output_img = np.zeros_like(img)
        output_img = output_img.transpose([0, 2, 1])    #[C, W, H]

        for i in range(shape_y):        #(0, 1, 2)
            for j in range(shape_x):    #(0, 1)
                if i == 0:
                    if j == 0:
                        center_x, center_y = 1024, 1024
                    elif j == shape_x - 1:
                        center_x, center_y = width - 1024, 1024
                    else:
                        center_x, center_y = width // 2, 1024

                elif i == shape_y - 1: 
                    if j == 0:
                        center_x, center_y = 1024, height - 1024
                    elif j == shape_x - 1:
                        center_x, center_y = width - 1024, height - 1024
                    else:
                        center_x, center_y = width // 2, height - 1024
                else:
                    if j == 0:
                        center_x, center_y = 1024, height // 2
                    elif j == shape_x - 1:
                        center_x, center_y = width - 1024, height // 2
                    else:
                        center_x, center_y = width // 2, height // 2

                output_img[:, center_x - 1024:center_x + 1024, center_y - 1024:center_y + 1024] = process_img_list[i][j].transpose(2, 1, 0)
        
        output_img = output_img.transpose(2, 1, 0).astype(np.uint8)
    return output_img

In [4]:
"""裁剪img"""
def img_crop(img):  
    CROP_SIZE = 2048
    '''
    input: ndarray, [C, H, W]
    output: 2048x2048, img_list(np.array)
    '''
    h, w = img.shape[1:]
    x_number = w // 2048 + 1
    y_number = h // 2048 + 1

    if (x_number, y_number) == (1, 1):
        return [[img]]

    else:
        img_list = []
        for i in range(y_number):
            img_list.append([])
        
        for i in range(1, y_number+1):
            for j in range(1, x_number+1):
                if i == 1:
                    center_y = 1024
                elif i == y_number :
                    center_y = h - 1024
                else:
                    center_y = (h - 2048) // (y_number - 1) * (i-1) + 1024
                
                if j == 1:
                    center_x = 1024
                elif j == x_number:
                    center_x = w - 1024
                else:
                    center_x = (w - 2048) // (x_number - 1) * (j-1) + 1024

                img1 = img[:, center_y-1024:center_y+1024, center_x-1024:center_x+1024]

                img_list[i-1].append(img1)      #[C, H, W]

        return img_list

In [1]:
"""预测"""
"""保留印刷区域小部分"""
def predict(img_path, model):
    image_name = img_path
    ori_img = Image.open(image_name)    #[H, W, C]
    ori_img = np.array(ori_img).transpose(2, 0, 1)      #[C, H, W]
    ori_img0, ori_img1, ori_img2 = ori_img[0], ori_img[1], ori_img[2]   #R, G, B
    ori_img00, ori_img11, ori_img22 = ori_img0.copy(), ori_img1.copy(), ori_img2.copy()

    result = model.predict(image_name)

    img = result['label_map']

    #去除手写
    tmp_img = img.copy().astype(np.uint8)   #去除手写
    tmp_img2 = img.copy().astype(np.uint8)  #保留印刷
    tmp_img[tmp_img == 0] = 0
    tmp_img[tmp_img == 1] = 255     #手写
    tmp_img[tmp_img == 2] = 0
    kernel = np.ones((2, 2), np.uint8)
    tmp_img = cv2.dilate(tmp_img, kernel, iterations = 1).astype(np.uint8)
    tmp_img[tmp_img >= 150] = 255

    #保留印刷
    img[img == 0] = 0
    img[img == 2] = 255     #印刷
    img[img == 1] = 0
    img = img.astype(np.uint8)
    kernel = np.ones((4, 4), np.uint8)
    kernel2 = np.ones((2, 4), np.uint8)
    img2 = cv2.dilate(img, kernel2, iterations = 1).astype(np.uint8)
    img = cv2.dilate(img, kernel, iterations = 4).astype(np.uint8)

    img[img >= 150] = 255
    img2[img2 >= 150] = 255

    retain_area = img - tmp_img
    retain_area[retain_area >= 255] = 255
    retain_area[img2 == 255] = 255

    ori_img0[retain_area == 255] = ori_img00[retain_area == 255]
    ori_img1[retain_area == 255] = ori_img11[retain_area == 255]
    ori_img2[retain_area == 255] = ori_img22[retain_area == 255]
    ori_img0[retain_area != 255] = 255
    ori_img1[retain_area != 255] = 255
    ori_img2[retain_area != 255] = 255
    ori_img = np.array([ori_img0, ori_img1, ori_img2]).transpose(1, 2, 0).astype(np.uint8)  #[H, W, C]

    #去除红色
    ori_img = ori_img.transpose(2, 0, 1)
    ori_img0, ori_img1, ori_img2 = ori_img[0], ori_img[1], ori_img[2]   #R, G, B
    tmp_img = ori_img.copy().astype(np.float32)
    tmp_img1 = tmp_img[0] - tmp_img[1] + tmp_img[0] - tmp_img[2]

    ori_img0[tmp_img1 >= 100] = 255
    ori_img1[tmp_img1 >= 100] = 255
    ori_img2[tmp_img1 >= 100] = 255
    ori_img = np.array([ori_img0, ori_img1, ori_img2]).transpose(1, 2, 0).astype(np.uint8)  #[H, W, C]

    return ori_img

In [8]:
"""预测"""
"""alldata2/best_model"""
model = pdx.load_model('model/deeplab_augument_alldata2/best_model')
if not os.path.exists('resultB1/'):
    os.makedirs('resultB1/')

for index in range(1, 201): 
    dir_path = 'data/dehw_testB_dataset/dehw_testB_'
    path = f'{index}'
    img = erase_hand_write(dir_path + path + '.jpg', model)

    img = Image.fromarray(img)
    img.save('resultB1/' + 'dehw_testB_' + path + '.png')

    print(f'已完成：{index}张图片')

W0121 01:21:27.804330  5206 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0121 01:21:27.808977  5206 device_context.cc:465] device: 0, cuDNN Version: 7.6.


2022-01-21 01:21:30 [INFO]	Model[DeepLabV3P] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


已完成：1张图片
已完成：2张图片
已完成：3张图片
已完成：4张图片
已完成：5张图片
已完成：6张图片
已完成：7张图片
已完成：8张图片
已完成：9张图片
已完成：10张图片
已完成：11张图片
已完成：12张图片
已完成：13张图片
已完成：14张图片
已完成：15张图片
已完成：16张图片
已完成：17张图片
已完成：18张图片
已完成：19张图片
已完成：20张图片
已完成：21张图片
已完成：22张图片
已完成：23张图片
已完成：24张图片
已完成：25张图片
已完成：26张图片
已完成：27张图片
已完成：28张图片
已完成：29张图片
已完成：30张图片
已完成：31张图片
已完成：32张图片
已完成：33张图片
已完成：34张图片
已完成：35张图片
已完成：36张图片
已完成：37张图片
已完成：38张图片
已完成：39张图片
已完成：40张图片
已完成：41张图片
已完成：42张图片
已完成：43张图片
已完成：44张图片
已完成：45张图片
已完成：46张图片
已完成：47张图片
已完成：48张图片
已完成：49张图片
已完成：50张图片
已完成：51张图片
已完成：52张图片
已完成：53张图片
已完成：54张图片
已完成：55张图片
已完成：56张图片
已完成：57张图片
已完成：58张图片
已完成：59张图片
已完成：60张图片
已完成：61张图片
已完成：62张图片
已完成：63张图片
已完成：64张图片
已完成：65张图片
已完成：66张图片
已完成：67张图片
已完成：68张图片
已完成：69张图片
已完成：70张图片
已完成：71张图片
已完成：72张图片
已完成：73张图片
已完成：74张图片
已完成：75张图片
已完成：76张图片
已完成：77张图片
已完成：78张图片
已完成：79张图片
已完成：80张图片
已完成：81张图片
已完成：82张图片
已完成：83张图片
已完成：84张图片
已完成：85张图片
已完成：86张图片
已完成：87张图片
已完成：88张图片
已完成：89张图片
已完成：90张图片
已完成：91张图片
已完成：92张图片
已完成：93张图片
已完成：94张图片
已完成：95张图片
已完成：96张图片
已完成：97张图片
已完成：98张图片
已完成：99张图片
已完成：100张图片
已完成：101张

In [9]:
%cd resultB1/
!zip -D resultB1.zip ./*

/home/aistudio/resultB1
  adding: dehw_testB_100.png (deflated 3%)
  adding: dehw_testB_101.png (deflated 5%)
  adding: dehw_testB_102.png (deflated 4%)
  adding: dehw_testB_103.png (deflated 4%)
  adding: dehw_testB_104.png (deflated 4%)
  adding: dehw_testB_105.png (deflated 4%)
  adding: dehw_testB_106.png (deflated 3%)
  adding: dehw_testB_107.png (deflated 3%)
  adding: dehw_testB_108.png (deflated 4%)
  adding: dehw_testB_109.png (deflated 7%)
  adding: dehw_testB_10.png (deflated 4%)
  adding: dehw_testB_110.png (deflated 5%)
  adding: dehw_testB_111.png (deflated 8%)
  adding: dehw_testB_112.png (deflated 6%)
  adding: dehw_testB_113.png (deflated 6%)
  adding: dehw_testB_114.png (deflated 4%)
  adding: dehw_testB_115.png (deflated 5%)
  adding: dehw_testB_116.png (deflated 3%)
  adding: dehw_testB_117.png (deflated 4%)
  adding: dehw_testB_118.png (deflated 2%)
  adding: dehw_testB_119.png (deflated 4%)
  adding: dehw_testB_11.png (deflated 4%)
  adding: dehw_testB_120.png (de